In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
from scipy.special import iv
from statsmodels.tsa.arima.model import ARIMA
import altair
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import expm

In [2]:
def mets_filter(ts,rho,alpha):
    ts_mean = np.mean(ts)
    ts = np.append(ts,ts_mean)
    sample_size = len(ts)
    
    L_approx = csc_matrix( np.zeros([sample_size, sample_size]) )

    for row in range(0,24):
        L_approx[row,row+1] =1

    filter_matrix = expm(rho*L_approx)
    
    output = np.dot(filter_matrix,ts) + alpha

    return output[:-1]


def ols_mets(ts, grid_size):
    sample_size = len(ts)
    L_approx = np.zeros([sample_size, sample_size])

#     for row in range(0,len(L_approx)-1):
#         L_approx[row][row+1] =1

    for row in range(1,len(L_approx)):
        L_approx[row][row-1] =1
    
    min_rho =  0.0
    filter_matrix = expm(min_rho*L_approx)
    resid = np.dot(filter_matrix,ts)
    min_alpha =  np.mean(resid)
    min_obj = np.square(np.std(resid))
    
    for rho in np.linspace(-2,2,grid_size):
        filter_matrix = expm(rho*L_approx)
        resid =  np.dot(filter_matrix,ts)
        alpha =  np.mean(resid)
        obj = np.square(np.std(resid))
        
        if obj < min_obj:
            min_alpha = alpha
            min_rho = rho
            min_obj = obj

    return [min_alpha, min_rho, min_obj]    

In [3]:
data = pd.read_excel('C:/Users/michelj8/Documents/GitHub/METS/fred_md.xlsx')
data = data.set_index('sasdate')
series_list = data.columns
data.head()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
Transform:,5.000,5.0,5.000,5.0000,5.00000,5.0000,5.0000,5.0000,5.0000,5.0000,...,6.000,6.00,6.00,6.00,2.0,6.0,6.0,6.0,6.0000,1.0
1959-01-01 00:00:00,2437.296,2288.8,17.302,292258.8329,18235.77392,22.6250,23.4581,22.1904,32.4078,21.9882,...,11.358,2.13,2.45,2.04,NaN,274.9,6476.0,12298.0,84.2043,NaN
1959-02-01 00:00:00,2446.902,2297.0,17.482,294429.5453,18369.56308,23.0681,23.7747,22.3827,32.6455,22.1036,...,11.375,2.14,2.46,2.05,NaN,276.0,6476.0,12298.0,83.5280,NaN
1959-03-01 00:00:00,2462.689,2314.0,17.647,293425.3813,18523.05762,23.4004,23.9186,22.4925,32.6455,22.5365,...,11.395,2.15,2.45,2.07,NaN,277.4,6508.0,12349.0,81.6405,NaN
1959-04-01 00:00:00,2478.744,2330.3,17.584,299331.6505,18534.46600,23.8989,24.2641,22.8221,33.1606,22.6807,...,11.436,2.16,2.47,2.08,NaN,278.1,6620.0,12484.0,81.8099,NaN


In [4]:
ar_mse = []
mets_mse  = []
series_type = []

ar_coef = []
mets_coef = []

# start_index = 493 #2000+
start_index = 583 #July 2007+

# for series in series_list[0:5]:
for series in series_list:
    transform = data[series][0]
    series_type.append(transform)
    
    if transform == 1:
        ts = data[series].iloc[start_index:].dropna().values
    if transform == 2:
        ts = (data[series].iloc[start_index:]).diff(1).dropna().values
    if transform == 3:
        ts = (data[series].iloc[start_index:]).diff(2).dropna().values
    if transform == 4:
        ts = np.log(data[series].iloc[start_index:]).dropna().values
    if transform == 5:
        ts = np.log(data[series].iloc[start_index:]).diff(1).dropna().values
    if transform == 6:
        ts = np.log(data[series].iloc[start_index:]).diff(2).dropna().values
    if transform == 7:
        ts = (data[series].iloc[start_index:]).pct_change(1).dropna().values
        
    mets_model = ols_mets(ts, 200)
    mets_mse.append(mets_model[2] )
    mets_coef.append(-1*mets_model[1])
    
    ar_model = ARIMA(ts,order = (1,0,0)).fit()
    ar_mse.append(np.mean(np.square(ts- ar_model.predict())))
    ar_coef.append(ar_model.params[1])

C:\Users\michelj8\Anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\michelj8\Anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\michelj8\Anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\michelj8\Anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\michelj8\Anaconda3\lib\site-packages\statsmodels\base\model.py:567: Converg

In [5]:
total_df = pd.DataFrame({'mets_mse':mets_mse, 
                              'ar_mse':ar_mse,
                              'transform':series_type,
                             'ar_coef':ar_coef,
                             'mets_coef':mets_coef})

total_df['MSE_Ratio'] =  100*(total_df['mets_mse']-total_df['ar_mse'])/total_df['ar_mse']
total_df['mets_beats_ar'] = 1*(total_df['MSE_Ratio'] < 0)
total_df['transform']=total_df['transform'].astype('int').astype('str')
total_df.head()

,mets_mse,ar_mse,transform,ar_coef,mets_coef,MSE_Ratio,mets_beats_ar
0,0.000145,0.000148,5,-0.289859,-0.351759,-1.619378,1
1,0.000081,0.000081,5,0.135231,0.150754,-0.331912,1
2,0.000210,0.000211,5,0.107936,0.170854,-0.591882,1
3,0.000216,0.000216,5,0.038173,0.050251,-0.047383,1
4,0.000503,0.000504,5,0.055665,0.070352,-0.127776,1


In [6]:
round(100*total_df['mets_beats_ar'].mean(),1)

64.8

In [7]:
total_df[['transform','mets_beats_ar']].groupby('transform').mean().apply(lambda s: round(100.0*s,1))

,mets_beats_ar
transform,
1,0.0
2,57.9
4,0.0
5,71.7
6,100.0
7,0.0


In [8]:
for trans in sorted(total_df['transform'].unique()):
    print( trans, len( total_df.query('transform == @trans')))

1 11
2 19
4 10
5 53
6 34
7 1


In [9]:
(altair.Chart(total_df, title = 'FRED-MD fitted coefficients')
 .mark_point()
 .encode(x=altair.X('ar_coef', title = 'AR(1) Coefficient'),
         y=altair.Y('mets_coef',title = 'METS Coefficient'),
         color=altair.Color('transform', title = 'Transform' )) )
#  .properties(
#     width=180,
#     height=180)
# )

alt.Chart(...)

In [10]:
(altair.Chart(total_df, title = 'FRED-MD normalized MSE').mark_bar().encode(
    altair.X("MSE_Ratio", bin=altair.Bin(extent=[-100, 100], step=25), title = 'Normalized difference in MSE'),
    y='count()',
         color=altair.Color('transform', title = 'Transform' )) ) 

alt.Chart(...)

In [11]:
(altair.Chart(total_df.query('MSE_Ratio < 100'), title = 'FRED-MD MSE comparison by METS coefficient')
 .mark_point()
 .encode(x=altair.X('mets_coef', title = 'METS Coefficient'),
         y=altair.Y('MSE_Ratio',title = 'Normalized difference in MSE'),
         color=altair.Color('transform', title = 'Transform' )) ) 

alt.Chart(...)

In [12]:
len(total_df)

128